# Building a reputation score on the Blockchain for Bitcoin

### This is in intro that will describe our workflow process. Below you will find detailed steps on our thought process and how we are implemeting our project. 

##### Getting set up 
** Step 1: Open both this notebook and the BigQuery DB **

** Step 2: Import the Lib's & insert the BigQuery project ID **

##### Creating the DB using SQL & BigQuery 
** Step 3: Define the exchanges which will be used as a baseline for flagging **

** Step 4: Map each address and save it as a table to the BigQuery DB ** 

##### Setup middleware (decision tree) 
** Will take an input (address) and based on the flagging that is set up spit out a reputation score **


In [1]:
# Import Lib's 
import pandas as pd
from pandas.io import gbq
import random

In [2]:
# Insert the BigQuery Project ID
projectid = ""

### Define the exchanges

In [5]:
# Mt.Gox Hack 
mt_Gox_Hack = gbq.read_gbq('SELECT * FROM (FLATTEN(FLATTEN(Capstone_Dataset.transactions_copy, inputs.input_pubkey_base58_error), outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1LNWw6yCxkUmkhArb2Nf2MPw6vG7u5WG7q"', projectid)


### Map the transaction history for a single transactor (address)

Step 1. Pick a random address to start with.

Step 2. Run a query that pulls the timestamp, transaction_id, input_pubkey, output_satoshis, and output_pubkey.

Step 3. For each output_pubkey in the quiered input_key, run a sparate query.

**Example, address1 has a pubkey of "19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4" whch returns five differnet pubkeys that it has sent coins to. The first being "1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9". Now, address2 is a query ran for "1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9". The second being "1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP". Now, address3 is a query ran for "1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP". And so on.**

Step 4. Once an adress is completly mapped, comment it out so we know it is done.

 ** IMPORTANT: after each query is ran, go to the BigQuery DB and save the query as a table and then proceed to the next query. The name of the table should be the pubkey **


In [28]:
# address1 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4"', projectid)
# address1.head()


,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,341933,1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9
1,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,394786,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP
2,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,339724,1137y8mRW7eeq8Yd6rbhG9Jb24JLRTcJMD
3,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,393127,1MQxu5aYY8sWeed8jRZQVBV5p8NaJqZYZL
4,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,5839000,15B5JixcpuVvF8XeDG8yenMwBeoZrrcnhQ


In [24]:
# address2 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9"', projectid)
# address2.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440622647000,c29bf82a125ee695deb10e2ce26b9029b8b837aa080a52...,1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9,1900000000,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y
1,1440622647000,c29bf82a125ee695deb10e2ce26b9029b8b837aa080a52...,1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9,1958297462,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL


In [26]:
# address3 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP"', projectid)
# address3.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440623706000,185c8ab7f1f642961c890e79bfb97675603e08b7f830d1...,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP,7900000,1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky
1,1440623706000,185c8ab7f1f642961c890e79bfb97675603e08b7f830d1...,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP,783994,1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs
2,1440623706000,185c8ab7f1f642961c890e79bfb97675603e08b7f830d1...,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP,3724150,19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr
3,1440623706000,185c8ab7f1f642961c890e79bfb97675603e08b7f830d1...,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP,147039844,1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV
4,1440623706000,185c8ab7f1f642961c890e79bfb97675603e08b7f830d1...,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP,7116006,1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9


In [29]:
# address4 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1137y8mRW7eeq8Yd6rbhG9Jb24JLRTcJMD"', projectid)
# address4.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440622647000,c29bf82a125ee695deb10e2ce26b9029b8b837aa080a52...,1137y8mRW7eeq8Yd6rbhG9Jb24JLRTcJMD,1900000000,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y
1,1440622647000,c29bf82a125ee695deb10e2ce26b9029b8b837aa080a52...,1137y8mRW7eeq8Yd6rbhG9Jb24JLRTcJMD,1958297462,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL


In [30]:
# address5 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1MQxu5aYY8sWeed8jRZQVBV5p8NaJqZYZL"', projectid)
# address5.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440622647000,c29bf82a125ee695deb10e2ce26b9029b8b837aa080a52...,1MQxu5aYY8sWeed8jRZQVBV5p8NaJqZYZL,1900000000,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y
1,1440622647000,c29bf82a125ee695deb10e2ce26b9029b8b837aa080a52...,1MQxu5aYY8sWeed8jRZQVBV5p8NaJqZYZL,1958297462,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL


In [31]:
# address6 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "15B5JixcpuVvF8XeDG8yenMwBeoZrrcnhQ"', projectid)
# address6.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440624905000,5af17ea2093e5771b2b26f46e4465919146440980672ad...,15B5JixcpuVvF8XeDG8yenMwBeoZrrcnhQ,944400000,1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM
1,1440624905000,5af17ea2093e5771b2b26f46e4465919146440980672ad...,15B5JixcpuVvF8XeDG8yenMwBeoZrrcnhQ,2085706576,1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq


In [32]:
address7 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y"', projectid)
address7.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440623020000,61548284eb7e435eb0562ad69b8c85d2cbd93dd7334069...,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y,124238400,1N31ke4y2fLAhUmBiu1s7eEV1XrwfbikUe
1,1440623020000,61548284eb7e435eb0562ad69b8c85d2cbd93dd7334069...,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y,54071626,1Fs5SxMw1S9tssShyGTD5u4Mx85tkN6kT9
2,1440623020000,61548284eb7e435eb0562ad69b8c85d2cbd93dd7334069...,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y,68440000,1Rcw9GmRov5wj8eyEfWTkBnD1zw4oWb6T
3,1440623020000,61548284eb7e435eb0562ad69b8c85d2cbd93dd7334069...,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y,2143921600,14Jb6VdPp4Z19YSpPJokfembo9q1yQ4PGh
4,1440623020000,61548284eb7e435eb0562ad69b8c85d2cbd93dd7334069...,14jqcZynaQB9jKRYWewhiszSVFi7jvDv1y,39468809,14L9xHuWayB76rcrKKEb61rXe2ehGwmDzf


In [33]:
address8 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL"', projectid)
address8.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440623403000,8dd237058e459d619b82179278751dde493d2b6cc45ebd...,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL,775000000,1AFjkqG14hH292wvo9VxQ8zDtmb5or8JhA
1,1440623403000,8dd237058e459d619b82179278751dde493d2b6cc45ebd...,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL,43841600,1LsA2HCTiTWehVqfPxQwidxfnXXKdAtuNj
2,1440623403000,8dd237058e459d619b82179278751dde493d2b6cc45ebd...,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL,1802230000,1EwJwBR3DNxEzn4Hu8wZw7JPu8pjmmW6BF
3,1440623403000,8dd237058e459d619b82179278751dde493d2b6cc45ebd...,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL,1480060000,1kPJx5E7UPNET9TA9Z5n6ozS2JjVkAYqB
4,1440623403000,8dd237058e459d619b82179278751dde493d2b6cc45ebd...,1JFXzksGGqFJBsg2kCH8Pa4xSt8cFvMbZL,2520000000,18SJMNpzHSqkCC136kcEywYZjvMEu37SEC


In [34]:
address9 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky"', projectid)
address9.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440624905000,2363d792958aa9537cc42e551e9b365ec5a3983b0c0748...,1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky,2630000000,1uJ9zdcC3GJ2WtzBe289DfHSb5XSsGT2q
1,1440624905000,2363d792958aa9537cc42e551e9b365ec5a3983b0c0748...,1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky,1974663426,1GYfZMh9egc3s7BrMAtj1ZsqwV5Z4gKNQ
2,1440624905000,2363d792958aa9537cc42e551e9b365ec5a3983b0c0748...,1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky,1974621877,1UZhXUTc6GLorgTRGE8bFzD4X1W5TgWeA
3,1440624905000,2363d792958aa9537cc42e551e9b365ec5a3983b0c0748...,1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky,304979,1KmBK7AQw9eaRgMuLXW44VhjSAphzgbZkL
4,1440624905000,2363d792958aa9537cc42e551e9b365ec5a3983b0c0748...,1DWWc9PzWBrHkABEsqWwgcYK6YD1RSjHky,3672497316,1L7LcXFJvfhgkM32FkDmNaPDJpA8pqEXPz


In [35]:
address10 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs"', projectid)
address10.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440623706000,497d9cdf6f645c6b349a43f28633f03edf98be1571d08c...,1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs,100000000,16P7mAsqRBraX5oUnXF6NFQ7k2bksJ1CAX
1,1440623706000,497d9cdf6f645c6b349a43f28633f03edf98be1571d08c...,1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs,108959844,1J6QW21bDpoHi8zELveF9BsRsQ28XD1RYf
2,1440623706000,497d9cdf6f645c6b349a43f28633f03edf98be1571d08c...,1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs,120840000,19FAsp2JxTboALeu5rDqfHfttQPec8cigi
3,1440623706000,497d9cdf6f645c6b349a43f28633f03edf98be1571d08c...,1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs,109066283,1JBBb7puA5W9SW22XXVQH8MQJGn23boTZ2
4,1440623706000,497d9cdf6f645c6b349a43f28633f03edf98be1571d08c...,1JK8CboYEkApaCCNVMCZ5zX2xbF8D619xs,47160000,1MdQbEEMZAFfyihEm5StvTHJyB1smNpxrK


In [36]:
address11 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr"', projectid)
address11.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440861523000,8e2031ec24a0ff8f8a048d9286ed99c0c1dbdab28494da...,19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr,108990000,14M4RASiZn57HpiN1hJWZz8upg2cJkWbbF
1,1440861523000,8e2031ec24a0ff8f8a048d9286ed99c0c1dbdab28494da...,19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr,108990000,14M4RASiZn57HpiN1hJWZz8upg2cJkWbbF
2,1440861523000,8e2031ec24a0ff8f8a048d9286ed99c0c1dbdab28494da...,19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr,39820618,1AguV89hBuXnHGYV87pBqPB8gQFXacrJb1
3,1440861523000,8e2031ec24a0ff8f8a048d9286ed99c0c1dbdab28494da...,19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr,39820618,1AguV89hBuXnHGYV87pBqPB8gQFXacrJb1
4,1440861523000,8e2031ec24a0ff8f8a048d9286ed99c0c1dbdab28494da...,19DcaHpsW4sLeraXFEq1yzZaDBZeiJTvTr,66650000,1FiVqSWMSs1vBZMF5iMaWTzSQhZt8Uvwgx


In [37]:
address12 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV"', projectid)
address12.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440623706000,1ca4d322709106956813cbde582a87e5240600047ffd26...,1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV,3060040000,16UPG6jiWssGf5n91QkNR5tShpCFr1NEFa
1,1440623706000,1ca4d322709106956813cbde582a87e5240600047ffd26...,1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV,3723020156,1MPrKx6Q9m7gyXqn7EckEQeoG1KMT2ipun
2,1440623706000,1ca4d322709106956813cbde582a87e5240600047ffd26...,1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV,868385362,1LZxDVXP8uupQY67a4HJ9HmUGbeV7ALm8d
3,1440623706000,1ca4d322709106956813cbde582a87e5240600047ffd26...,1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV,2248040000,1DZJ6eeDW44571AQTyVYK2p4dmFMmx8TAF
4,1440623706000,1ca4d322709106956813cbde582a87e5240600047ffd26...,1J8pKizgUeMi4vzeswdpWRaj6cmxWmX2uV,102900000,1Fye64mLETKs4T7FX97jpSs7cUHDgGGa1z


In [38]:
address13 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9"', projectid)
address13.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440624567000,51a24c46ec164f6d407e99325b47b9d630667084a6d213...,1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9,6040000,1GYp3NX1k5qvFe3gNzDzxojuEsGvxYM15U
1,1440624567000,51a24c46ec164f6d407e99325b47b9d630667084a6d213...,1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9,11880000,1NKW2wHcLFf28rXQi4WMRB6bJw21eYJGZw
2,1440624567000,51a24c46ec164f6d407e99325b47b9d630667084a6d213...,1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9,44000000,1CRqooy2acbCWJxQ5MqRWBTkzfAcyMS659
3,1440624567000,51a24c46ec164f6d407e99325b47b9d630667084a6d213...,1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9,17420000,1P2SK1CkVkkUofRoPoqMgApgDGTMbKkBe7
4,1440624567000,51a24c46ec164f6d407e99325b47b9d630667084a6d213...,1JW4EQmctnPeT5SperRKCjmpGKaL5YFKB9,22600000,1G7hFLZESfQajE3RDvfeuXd7CmnJorWcTF


In [41]:
address14 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq"', projectid)
address14.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440626675000,b86a298dec728fde64a91d2dee40a6b727ff6d12adf915...,1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq,366187371,1JmJVmLnJGgxSiEjNGZ53fp6hMwvyPHB6s
1,1440626675000,b86a298dec728fde64a91d2dee40a6b727ff6d12adf915...,1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq,1461570000,12ohNKamzL46m61v97dLuvzYQM2RrnHc4x
2,1440626675000,b86a298dec728fde64a91d2dee40a6b727ff6d12adf915...,1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq,308451331,1GfqmYGqDLfRZFvZetWVtjR9iAfjhDNFdZ
3,1440626675000,b86a298dec728fde64a91d2dee40a6b727ff6d12adf915...,1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq,1118131,147ei7NhLZEDFG7JBAJjwhXc58Dy6Ea4ZK
4,1440626675000,b86a298dec728fde64a91d2dee40a6b727ff6d12adf915...,1N3Tf4Nfn9mZ7Z5rCtJ3ahcZLuspDddXHq,491499205,1Jb7QrxbMxJaveSC3JA1Y7syjACpJBt4cQ


In [40]:
address15 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(Capstone_Dataset.transactions_copy, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM"', projectid)
address15.head()

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
0,1440625236000,a4f343463b71b576c099f2383dcf7994a475667563ce34...,1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM,15640000,1N5QQ9h888xsQtHS9Bvskqy4r37jMFM79Q
1,1440625236000,a4f343463b71b576c099f2383dcf7994a475667563ce34...,1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM,1536823,146BjscQAmV2vfTUwqwambWMmzwRWdazHf
2,1440625236000,a4f343463b71b576c099f2383dcf7994a475667563ce34...,1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM,1500359,1CvtagVNspMi6pnedj5mCg2jEwUxEUe7Pz
3,1440625236000,a4f343463b71b576c099f2383dcf7994a475667563ce34...,1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM,1640000,1CsH53Eovqp3Y5QAJxw36kyDmGssNFtjsi
4,1440625236000,a4f343463b71b576c099f2383dcf7994a475667563ce34...,1M2nS1PaqF8pjbLHB8M7GcGH99f7fgqnAM,4240000,14sKJaPHzowjzBhLFXfn1y7e4KJSiSTuN6


### Creating transaction history per address
Group by public key (input), then build the transactions on that public key.
A Bitcoin address is an identifier of 26-35 alphanumeric characters that represents a possible destination for a bitcoin payment. 

In [8]:
## First select a random public key from the data 
def randomPublic(df, n):
    return df.loc[random.sample(df.index, n)]

In [14]:
randomPublic(transaction, 1)

,timestamp,transaction_id,inputs_input_script_bytes,inputs_input_script_string,inputs_input_script_string_error,inputs_input_sequence_number,inputs_input_pubkey_base58,inputs_input_pubkey_base58_error,outputs_output_satoshis,outputs_output_script_bytes,...,outputs_output_script_string_error,outputs_output_pubkey_base58,outputs_output_pubkey_base58_error,block_id,previous_block,merkle_root,nonce,version,work_terahash,work_error
457574,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,3459000,"v�\tp� Z�ˤLf(\r�R8jR""鈬",...,None,1pwyXZZiQFPCr8Y2CsGV8xySYtVSMFJ8b,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None


In [10]:
## Start creating the tranaction history per address
#  map the adresses (Pay-to-PubkeyHash)
#  then build the tranactions on that public key

## pub with max at the top (rank)
def pubKey(df, x):
    return df.loc[lambda df: df.inputs_input_pubkey_base58 == x, :]

In [15]:
pubKey(transaction, '19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4')

,timestamp,transaction_id,inputs_input_script_bytes,inputs_input_script_string,inputs_input_script_string_error,inputs_input_sequence_number,inputs_input_pubkey_base58,inputs_input_pubkey_base58_error,outputs_output_satoshis,outputs_output_script_bytes,...,outputs_output_script_string_error,outputs_output_pubkey_base58,outputs_output_pubkey_base58_error,block_id,previous_block,merkle_root,nonce,version,work_terahash,work_error
457566,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,341933,v��� ��C��}S�c�x���{�䈬,...,None,1MsciWujD7AM7EuUDXDXBd7AgKZTBXU6v9,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457567,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,394786,v������z|)�0�6�|�6P�jĈ�,...,None,1Mb3nGCcfaKQiA4jSDsu38XYo13iphFhMP,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457568,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,339724,v� f�����JCa�� �<���g��,...,None,1137y8mRW7eeq8Yd6rbhG9Jb24JLRTcJMD,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457569,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,393127,v���xa��\�Y���h� d�[��,...,None,1MQxu5aYY8sWeed8jRZQVBV5p8NaJqZYZL,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457570,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,5839000,v�-�A� �c�Ԋ�&[��;� ;䈬,...,None,15B5JixcpuVvF8XeDG8yenMwBeoZrrcnhQ,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457571,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,6080000,v��P��9�����]﨧��Jֈ�,...,None,1FS6iAEnYRKeeRnzgXXvxa7Czia22J6eLj,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457572,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a62e675de...,None,4294967295,19HeuNTKobTgSbZyJf8sSHNEe5BchS1oW4,None,7280000,v�8��\n��)B�ɵ���f�O02��,...,None,169z9nQwwGXkWGEaw6H6GZ9MpdUM3NzJGn,None,000000000000000007a9c390f5ffedae8950d8ff36aac5...,0000000000000000103e01578611f0c285900545aae9c5...,5817fe41f56b0247a1716ea46b8f8510679196220b55ac...,1977788579,3,2330340086,None
457573,1440622265000,f4a3dcd4c841ea2964386caa9589c52731e129a0b21366...,H0E! �ox��4æ.g]��2�lFXa�-�/1Ak]�t' :}z��� ...,PUSHDATA(72)[3045022100c76f789fce34c3a6

In [68]:
transaction['inputs_input_pubkey_base58'].count()

417428

In [70]:
transaction['outputs_output_pubkey_base58'].count()

496707